In [54]:
import pandas as pd

In [55]:
df=pd.read_csv("data sets/hepatitis.csv")
df

,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,30,male,False,False,False,False,False,False,False,False,False,False,False,1.0,85.0,18.0,4.0,NaN,False,live
1,50,female,False,False,True,False,False,False,False,False,False,False,False,0.9,135.0,42.0,3.5,NaN,False,live
2,78,female,True,False,True,False,False,True,False,False,False,False,False,0.7,96.0,32.0,4.0,NaN,False,live
3,31,female,NaN,True,False,False,False,True,False,False,False,False,False,0.7,46.0,52.0,4.0,80.0,False,live
4,34,female,True,False,False,False,False,True,False,False,False,False,False,1.0,NaN,200.0,4.0,NaN,False,live
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,46,female,True,False,True,True,True,True,False,False,True,True,True,7.6,NaN,242.0,3.3,50.0,True,die
151,44,female,True,False,True,False,False,True,True,False,False,False,False,0.9,126.0,142.0,4.3,NaN,True,live
152,61,female,False,False,True,True,False,False,True,False,True,False,False,0.8,75.0,20.0,4.1,NaN,True,live
153,53,male,False,False,True,False,False,True,False,True,True,False,True,1.5,81.0,19.0,4.1,48.0,True,live


In [56]:
df.drop("protime",axis=1,inplace=True)

In [57]:
cat=df.select_dtypes(include="object").columns
cat

Index(['sex', 'steroid', 'fatigue', 'malaise', 'anorexia', 'liver_big',
       'liver_firm', 'spleen_palpable', 'spiders', 'ascites', 'varices',
       'class'],
      dtype='object')

In [58]:
df["steroid"]=df["steroid"].fillna(df["steroid"].mode()[0])
df["fatigue"]=df["fatigue"].fillna(df["fatigue"].mode()[0])
df["malaise"]=df["malaise"].fillna(df["malaise"].mode()[0])
df["anorexia"]=df["anorexia"].fillna(df["anorexia"].mode()[0])
df["liver_big"]=df["liver_big"].fillna(df["liver_big"].mode()[0])
df["liver_firm"]=df["liver_firm"].fillna(df["liver_firm"].mode()[0])
df["spleen_palpable"]=df["spleen_palpable"].fillna(df["spleen_palpable"].mode()[0])
df["spiders"]=df["spiders"].fillna(df["spiders"].mode()[0])
df["ascites"]=df["ascites"].fillna(df["ascites"].mode()[0])
df["varices"]=df["varices"].fillna(df["varices"].mode()[0])


In [59]:
num=df.select_dtypes(exclude="object").columns
num

Index(['age', 'steroid', 'antivirals', 'fatigue', 'malaise', 'anorexia',
       'liver_big', 'liver_firm', 'spleen_palpable', 'spiders', 'ascites',
       'varices', 'bilirubin', 'alk_phosphate', 'sgot', 'albumin',
       'histology'],
      dtype='object')

In [60]:
df["bilirubin"]=df["bilirubin"].fillna(df["bilirubin"].mean())
df["alk_phosphate"]=df["alk_phosphate"].fillna(df["alk_phosphate"].median())
df["sgot"]=df["sgot"].fillna(df["sgot"].median())
df["albumin"]=df["albumin"].fillna(df["albumin"].mean())

In [61]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in cat:
    df[col] = le.fit_transform(df[col])

In [62]:
X=df.drop("class",axis=1)
y=df["class"]

## our data is imbalance let's handle it first:

In [63]:
from imblearn.over_sampling import RandomOverSampler
os=RandomOverSampler()

In [64]:
X,y=os.fit_resample(X,y)

In [65]:
df1=pd.concat([X,y],axis=1)

In [66]:
X=df.drop("class",axis=1)
y=df["class"]

## feature selection

In [67]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)

In [68]:
from sklearn.feature_selection import mutual_info_classif
mutual_info=mutual_info_classif(X_train,y_train)
mutual_info

array([0.04205459, 0.00188686, 0.04553683, 0.0308284 , 0.10830562,
       0.0240159 , 0.06604574, 0.08013447, 0.        , 0.0124923 ,
       0.05350339, 0.10433626, 0.        , 0.09092582, 0.01433546,
       0.        , 0.15924799, 0.08941253])

In [69]:
mutual_info=pd.Series(mutual_info)
mutual_info.index=X_train.columns
mutual_info.sort_values(ascending=False)

albumin            0.159248
fatigue            0.108306
ascites            0.104336
bilirubin          0.090926
histology          0.089413
liver_big          0.080134
anorexia           0.066046
spiders            0.053503
steroid            0.045537
age                0.042055
antivirals         0.030828
malaise            0.024016
alk_phosphate      0.014335
spleen_palpable    0.012492
sex                0.001887
liver_firm         0.000000
varices            0.000000
sgot               0.000000
dtype: float64

In [70]:
from sklearn.feature_selection import SelectKBest
sel_5_cols=SelectKBest(mutual_info_classif,5)
sel_5_cols.fit(X_train,y_train)
X_train.columns[sel_5_cols.get_support()]

C:\Users\m\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass k=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Index(['liver_big', 'ascites', 'varices', 'bilirubin', 'albumin'], dtype='object')

In [71]:
cols=['age', 'bilirubin', 'alk_phosphate', 'sgot', 'albumin','class']

In [72]:
df2=df1[cols]
df2

,age,bilirubin,alk_phosphate,sgot,albumin,class
0,30,1.0,85.0,18.0,4.0,1
1,50,0.9,135.0,42.0,3.5,1
2,78,0.7,96.0,32.0,4.0,1
3,31,0.7,46.0,52.0,4.0,1
4,34,1.0,85.0,200.0,4.0,1
...,...,...,...,...,...,...
241,41,4.2,65.0,120.0,3.4,0
242,57,4.1,85.0,48.0,2.6,0
243,54,3.9,120.0,28.0,3.5,0
244,70,1.7,109.0,528.0,2.8,0


In [73]:
X=df1.drop("class",axis=1)
y=df1["class"]

In [74]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)

from sklearn import metrics



In [75]:
from sklearn.ensemble import RandomForestClassifier
model1=RandomForestClassifier()
model1.fit(X_train,y_train)
y_pred=model1.predict(X_test)
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        24
           1       1.00      0.88      0.94        26

    accuracy                           0.94        50
   macro avg       0.94      0.94      0.94        50
weighted avg       0.95      0.94      0.94        50



In [76]:
import pickle

In [77]:
filename='hepatitis.pkl'

In [78]:
pickle.dump(model1,open(filename,'wb'))